<a href="https://colab.research.google.com/github/KNS-Karthik/AITTA/blob/main/Unit_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# prompt: https://github.com/KNS-Karthik/AITTA.git
# clone this and save it

!git clone https://github.com/KNS-Karthik/AITTA.git


Cloning into 'AITTA'...


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
#Verify the file is copied to current directory
!ls -l

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
total 8
drwx------ 6 root root 4096 Mar  9 17:42 drive
drwxr-xr-x 1 root root 4096 Mar  6 14:29 sample_data


# 2(a) App for Speech recognition and Synthesis through APIs

In [ ]:
# Libraries required Speech recognition and Synthesis
!pip install SpeechRecognition
!pip install pocketsphinx
!pip install gTTS

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 51.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.2/29.2 MB 25.9 MB/s eta 0:00:00


In [ ]:
import speech_recognition as sr
# Initialize the recognizer
r = sr.Recognizer()
# Load your audio file
with sr.AudioFile("/content/drive/MyDrive/Dataset/Malevoice.wav") as source:
    audio = r.record(source) # read the entire audio file
# Recognize speech using Sphinx
try:
    print("Sphinx thinks you said:")
    print(r.recognize_sphinx(audio))
except sr.UnknownValueError:
    print("Sphinx could not understand audio")
except sr.RequestError as e:
    print(f"Sphinx error; {e}")

Sphinx thinks you said:
what if somebody did that the break it be careful that you keep adequate coverage but look for places that they might it be that taking longer to get the word away then the bankers expected hiring the lifer want company they went hurt bad and retirement


In [ ]:
from gtts import gTTS
import os # Removed the extra space before import os
text_to_say = "Hello, this is a simple text to speech conversion demonstration"
tts = gTTS(text=text_to_say, lang='en')
tts.save("output.mp3")
os.system("mpg321 output.mp3")

32512

# 2(b) Building a Chatbot

In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
# Download NLTK stopwords (if not already downloaded)
nltk.download('stopwords')
# Download the 'punkt_tab' resource
nltk.download('punkt_tab') # Download 'punkt_tab' resource
# Load the dataset into a DataFrame.
df = pd.read_csv('/content/drive/MyDrive/Dataset/electronics_faq_dataset.csv')
# Define a set of stopwords to be removed during text preprocessing
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # Tokenize the text
    words = word_tokenize(text)
    # Convert text to lowercase and remove stopwords
    words = [word.lower() for word in words if word.isalnum() and word.lower() not in stop_words]
    # Join the remaining words to form preprocessed text
    preprocessed_text = ' '.join(words)
    return preprocessed_text

def find_best_match(user_input):
    # Preprocess user input for better keyword matching.
    user_input = preprocess_text(user_input)
    # Initialize variables to keep track of the best match.
    best_match_score = 0
    best_answer = "Sorry, I couldn't find an answer to your question. Can you please specify or ask in a different way?"
    # Loop through each row in the dataset.
    for _, row in df.iterrows():
        # Preprocess keywords for better matching.
        keywords = preprocess_text(row['Keyword'])
        # Split keywords into a list of words.
        keyword_list = keywords.split()
        # Calculate the match score as the number of overlapping words between user input and keywords.
        match_score = len(set(user_input.split()) & set(keyword_list))
        # If the current row's match score is higher than the best one we've seen so far, update best_match_score and best_answer.
        if match_score > best_match_score:
            best_match_score = match_score
            best_answer = f"FAQ: {row['FAQ']}\nAnswer: {row['Answer']}"
    # Return the best answer found (or the default answer if no match was found).
    return best_answer

# Start the chatbot conversation loop.
while True:
    # Ask the user to type their question.
    user_question = input("Hello! How can I help you? Please type your question or type 'quit' to exit: ")
    # Check if the user wants to quit the chat.
    if user_question.lower() == 'quit':
        print("Thank you for chatting with us. Have a great day!")
        break  # Exit the chat loop.
    # Find the best response for the user's question and print it.
    response = find_best_match(user_question)
    print(response)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


Hello! How can I help you? Please type your question or type 'quit' to exit: quit
Thank you for chatting with us. Have a great day!


# 2(c) Build a virtual assistant

In [ ]:
# Library to be install
!pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11679 sha256=96c394247aa57840639d03e65df6674142c4b2f573515676784883e298f0ec8b
  Stored in directory: /root/.cache/pip/wheels/8f/ab/cb/45ccc40522d3a1c41e1d2ad53b8f33a62f394011ec38cd71c6
Successfully built wikipedia


In [ ]:
import wikipedia

def get_wikipedia_summary(query):
    try:
        # Set the language for Wikipedia (e.g., 'en' for English)
        wikipedia.set_lang('en')
        # Get the summary of the Wikipedia page
        summary = wikipedia.summary(query)
        return summary
    except wikipedia.exceptions.PageError:
        return "Sorry, I couldn't find a Wikipedia page for that query."
    except wikipedia.exceptions.DisambiguationError as e:
        options = e.options[:5]  # Limit to the first 5 options for simplicity
        return f"Multiple options found: {', '.join(options)}. Please specify your query."

if __name__ == "__main__":
    print("Hello! I'm your virtual assistant.")
    while True:
        user_input = input("Ask me about a topic, and I'll provide you with a summary. Type 'quit' to exit: ")
        if user_input.lower() == 'quit':
            print("Goodbye!")
            break
        response = get_wikipedia_summary(user_input)
        print(response)

Hello! I'm your virtual assistant.
Ask me about a topic, and I'll provide you with a summary. Type 'quit' to exit: quit
Goodbye!
